# Импортирование необходимых модулей

In [0]:
import vk
import getpass
import pandas as pd
from datetime import datetime
import time
import pickle
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords
import pymorphy2
from nltk.tokenize import RegexpTokenizer
from scipy.sparse import coo_matrix, vstack, hstack, csr_matrix
from sklearn.neighbors import NearestNeighbors
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn import metrics
import matplotlib.pyplot as plt


# Загрузка и фильтр постов

**Вход в ВК**

In [0]:
session = vk.AuthSession('6810913', raw_input(), getpass.getpass())
vk_api = vk.API(session, v = '2.033')

**Инициализация списка новостных групп**

In [0]:
#list of publics I take posts and comments from
public_list = [['РЕН ТВ', 26493942],['Лента.ру', 67991642], ['Медуза', 76982440],
               ['Коммерсантъ', 23482909], ['РБК', 25232578], ['РИА Новости', 15755094], ['Mail.ru Новости', 106879986],
               ['Газета.Ru', 20169232], ['Российская газета', 23304496], ['IZ.RU', 27532693], ['Московский Комсомолец (МК)', 20648295]]
#public_list = [['Московский Комсомолец (МК)', 20648295]]

**Загрузка постов из новостных групп**

In [0]:
%%time
def timestamp(time):
    """ transform timestamp given in seconds into %Y-%m-%d %H:%M:%S - form
    """
    return datetime.utcfromtimestamp(time).strftime('%Y-%m-%d %H:%M:%S')
#public_id = -25232578
#num_of_posts = 10000
#posts = vk_api.wall.get(owner_id = public_id, count = 100, offset = offset_value) #count - число (макс. 100), offset - сдвиг
def get_posts(posts, public_list):
    """ Function collecting posts from publics with ids from public_list into posts variable
        Input: posts - list (usually empty), public_list - list looking like [['name_of_public', id], ...]
        Output: list with posts from VK
    """
    for public_num in range(len(public_list)):
        offset_value = 0
        public_id = public_list[public_num][1] * (-1) #id is multiplicated by -1 because public id starts with "-"
        stop = True #flag stop
        while stop == True:
            to_extend = vk_api.wall.get(owner_id = public_id, count = 100, offset = offset_value)[1:] #count - число (макс. 100), offset - сдвиг
            if int(timestamp(to_extend[0]['date']).split('-')[0]) == 2018: #taking data beginning from year 2018
                if int(timestamp(to_extend[0]['date']).split('-')[1]) >= 6: #taking data beginning from month 6
                    posts.extend(to_extend)
                    time.sleep(1) #sleep is required otherwise account's banned
                    print(timestamp(to_extend[0]['date']))
                else:
                    stop = False
                    time.sleep(1)
                    print(timestamp(to_extend[0]['date']))
            else:
                posts.extend(to_extend)
                time.sleep(1)
                print(timestamp(to_extend[0]['date']))
            offset_value += 100 #increase offset to get futher 100 posts
    return posts

In [0]:
posts = []
#test of get_posts
posts = get_posts(posts, public_list[:1])



**Фильтрация постов**

In [0]:

def find_attachment(num):
    """ find whether there's an attachment to the post
    """
    try:
        return posts[i]['attachment']['link']['title']
    except:
        return '-'

In [0]:
#get only needed information from posts    
updated_posts = [[posts[i]['text'], find_attachment(i),
                posts[i]['marked_as_ads'], posts[i]['to_id'],
                 posts[i]['id'], posts[i]['likes']['count'], posts[i]['reposts']['count'], posts[i]['comments']['count'], timestamp(posts[i]['date'])]
                 for i in range(len(posts)) if posts[i][u'post_type'] == u'post']  

In [0]:
#transform updated_posts into dataframe
columns = ['text', 'attachment', 'marked_as_ads', 'to_id', 'id', 'likes', 'reposts', 'comments', 'data']
df = pd.DataFrame.from_records(updated_posts, columns = columns)
#df = df.append(df_new).reset_index(drop = True)
df.head()

,text,attachment,marked_as_ads,to_id,id,likes,reposts,comments,data
0,Волна отставок в американском правительстве до...,«Ошеломляющие нарушения этики»: за что Трамп у...,0,-20648295,486122,0,0,0,2018-12-16 14:52:24
1,"В Киеве состоялся Собор новой церкви, признанн...",В Киеве объявили о создании своей поместной це...,0,-20648295,486104,2,0,9,2018-12-16 14:33:14
2,Прелестным спектаклем порадовал публику Большо...,В Большом театре премьера оперы Россини «Путеш...,0,-20648295,486084,0,0,2,2018-12-16 14:04:33
3,Итог скандальной истории с российскими биатлон...,Пихлер нам поможет: Международный союз биатлон...,0,-20648295,486069,3,0,0,2018-12-16 13:47:23
4,В Приморском крае в данную минуту ведется акти...,Кожемяко сохранил лидерство после обработки по...,0,-20648295,486067,1,0,3,2018-12-16 13:45:17


In [0]:
#filter df
df = df[df.data <= '2018-10-01'][df.data >= '2017-10-01'][df.marked_as_ads == 0].drop('marked_as_ads', 1).reset_index(drop=True)
df.head()

**Сохранение постов**

In [0]:
#save in posts.pkl
with open('posts.pkl', 'wb') as f:
    pickle.dump(df, f)
    f.close()

# Предобработка текста

In [0]:
# Merge main text and attachment text
df['full_text'] = df.text + ' ' + df.attachment
df = df[['full_text', 'to_id', 'id', 'likes', 'reposts', 'data']]

In [0]:
def text_preprocess(series):
    """ Remove links, <*> and additional spaces from text
    """
    series_local = pd.DataFrame()
    series_local = series
    for text_num in range(len(series)):
        text = series_local[text_num]
        try:
          text = text.decode('utf-8')
        except:
          pass
        pattern = re.compile('<[^>]*>')
  #      text = pattern.sub(' ', text)
  #      pattern = re.compile('\(*\)')
        text = pattern.sub(' ', text)
        pattern = re.compile('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
        text = pattern.sub(' ', text)
        pattern = re.compile('\s+')
        text = pattern.sub(' ', text)
        text = ' '.join([i for i in text.split() if i[0] != '#' and i[0] != '@' and i[:5] != u'ВИДЕО' and i[:4] != u'ФОТО' and i[0] != '('])
        series_local[text_num] = text
    return series_local

In [0]:
df.full_text = text_preprocess(df.full_text[:df.shape[0]])

# Работа с TF-IDF

In [0]:
morph = pymorphy2.MorphAnalyzer()

import nltk
nltk.download('punkt')
nltk.download('stopwords')

from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

def pymorphy_lemmatizer(text):
  try:
    text = text.decode('utf-8')
  except:
    pass
  tokens = tokenizer.tokenize(text)
    # все буквы в словах заменяем на строчные
  tokens = [token.lower() for token in tokens]
    # удаляем цифры
  tokens = [re.sub(r'\d', '', i) for i in tokens]
  stop_words = stopwords.words('russian')
  tokens = [i for i in tokens if i not in stop_words]
    # удаляем слова, содержащие две или менее букв
  tokens = [i for i in tokens if not len(i) <= 2]
 # text_list = text.decode('utf-8').split()
  text_list = [morph.parse(x)[0].normal_form for x in tokens]
  return text_list

In [0]:
tfidf = TfidfVectorizer(tokenizer=pymorphy_lemmatizer)
tfs = tfidf.fit_transform(df.full_text[:df.shape[0]])

**Сохранение файла tfs**


In [0]:
#save in posts.pkl
with open('tfs.pkl', 'wb') as f:
    pickle.dump(df, f)
    f.close()

# Работа с датасетом yandex_rss_2016

**Инициализация датасета**

In [0]:
columns = ['id', 'short', 'long', 'label', 'date', 'source']
with open("yandex_rss_2016.csv", "rb") as f:
        df = pd.read_csv(f, names = columns, header=None)

In [0]:
df['both'] = df.short + ' ' + df.long
df = df[['id', 'both', 'label', 'date', 'source']]
df.head()

,id,both,label,date,source
0,0-0-Z0086E86AF725CE37,Mercedes X-Class? Базовая комплектация может б...,Пикап Mercedes-Benz получит имя X-Class,2016-01-26,Amsrus.ru
1,0-0-Z00BF7B02A7D0361E,ОПЕК призывает производителей нефти объединить...,ОПЕК предлагает остановить падающую нефть сообща,2016-01-26,Neftegaz.ru
2,0-0-Z00FC57A819528701,Е.Сыдыков избран председателем общественного с...,Состоялось Первое заседание общественного сове...,2016-01-26,Zakon.kz
3,0-0-Z01070795B458D465,Нелегальный бизнес по перепродаже безакцизного...,Канал поставки алкоголя из duty free перекрыли...,2015-12-23,Белинформ
4,0-0-Z01093EBA50C568F4,Л. Суарес: гол «Реалу» на «Сантьяго Бернабеу» ...,Луис Суарес назвал самый запоминающийся гол в ...,2015-12-23,Чемпионат


## Создание TF-IDF матрицы

In [0]:
morph = pymorphy2.MorphAnalyzer()
tokenizer = RegexpTokenizer(r'\w+')

def pymorphy_lemmatizer(text):
  try:
    text = text.decode('utf-8')
  except:
    pass
  tokens = tokenizer.tokenize(text)
    # все буквы в словах заменяем на строчные
  tokens = [token.lower() for token in tokens]
    # удаляем цифры
  tokens = [re.sub(r'\d', '', i) for i in tokens]
  stop_words = stopwords.words('russian')
  tokens = [i for i in tokens if i not in stop_words]
    # удаляем слова, содержащие две или менее букв
  tokens = [i for i in tokens if not len(i) <= 2]
 # text_list = text.decode('utf-8').split()
  text_list = [morph.parse(x)[0].normal_form for x in tokens]
  return text_list

In [0]:
tfidf = TfidfVectorizer(tokenizer=pymorphy_lemmatizer, ngram_range=(1,2))
tfs = tfidf.fit_transform(df.both[:df.shape[0]])

In [0]:
with open("tfs_yandex_rss.pkl", "wb") as f:
    pickle.dump(tfs, f)
    f.close()

In [0]:
with open('tfs_yandex_rss.pkl', 'rb') as f:
  tfs = pickle.load(f)

## Поиск оптимального параметра eps для кластеризатора DBSCAN

In [0]:
def dbscan_(eps):
  db = DBSCAN(eps=eps, min_samples=2, algorithm = 'brute', metric='cosine').fit(tfs)
  core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
  core_samples_mask[db.core_sample_indices_] = True
  labels = db.labels_

  df['cluster'] = labels
  tmp_df = []
  k_initial = 0
  k_end = 0
  tmp = df.sort_values('cluster').reset_index(drop=True)
  tmp = tmp[tmp.cluster != -1].reset_index(drop=True)
  for index in range(tmp.shape[0]):
    if index != tmp.shape[0]-1 and tmp.cluster[index + 1] == tmp.cluster[index]:
      k_end += 1
    else:
      if k_end - k_initial != 0:
        tmp_df.append(tmp[k_initial:k_end + 1].sort_values('label'))
      k_initial = k_end + 1
      k_end +=1


  shorted_list = []
  for tmp in tmp_df:
    list = tuple()
    for index in tmp.label.index:
          #list.append(tmp_df.label[index])
          #list.append(tmp_df.cluster[index])
      list = list + (tmp.label[index], tmp.cluster[index])
          #shorted_list.append((tmp_df.label[index], tmp_df.cluster[index]))
    shorted_list.append(tuple(list))
  set_clusters = set(shorted_list)


  tmp_df = []
  k_initial = 0
  k_end = 0
  tmp = df.sort_values('label').reset_index(drop=True)
  for index in range(tmp.shape[0]):
    if index != tmp.shape[0]-1 and tmp.label[index + 1] == tmp.label[index]:
      k_end += 1
    else:
      if k_end - k_initial != 0:
        tmp_df.append(tmp[k_initial:k_end + 1])
      k_initial = k_end + 1
      k_end +=1

  shorted_list = []
  for tmp in tmp_df:
    list = tuple()
    for index in tmp.label.index:
      list = list + (tmp.label[index], tmp.cluster[index])
    shorted_list.append(tuple(list))
  set_labels = set(shorted_list)


  label_cluster = float(len(set_labels.intersection(set_clusters)))/len(set_labels)
  print(eps, label_cluster)
  return label_cluster, set_clusters, set_labels

In [0]:
label_cluster_list = []
for k in range(1, 50):
  label_cluster_list.append(dbscan_(0.02 * k))

(0.02, 0.002847988608045568)
(0.04, 0.003737985048059808)
(0.06, 0.004746647680075947)
(0.08, 0.005458644832087339)
(0.1, 0.006645306752106325)
(0.12, 0.008603298920137652)
(0.14, 0.010976622760175626)
(0.16, 0.014892607096238282)
(0.18, 0.018511925952296192)
(0.2, 0.02379257149638068)
(0.22, 0.028835884656461374)
(0.24, 0.036430520944582885)
(0.26, 0.04598314940073573)
(0.28, 0.05517977928088288)
(0.3, 0.06686839919306989)
(0.32, 0.07867568529725881)
(0.34, 0.09066097068945057)
(0.36, 0.10418891657766702)
(0.38, 0.11516553933784265)
(0.4, 0.12940548237807048)
(0.42, 0.1425180965942803)
(0.44, 0.15450338198647207)
(0.46, 0.16435267592262964)
(0.48, 0.17058265100272932)
(0.5, 0.17627862821882045)
(0.52, 0.1799572801708793)
(0.54, 0.17515129939480242)
(0.56, 0.16856532573869704)
(0.58, 0.1587753648985404)
(0.6, 0.14506941972232112)
(0.62, 0.13106680906609708)
(0.64, 0.11391954432182272)
(0.66, 0.09428028954550849)
(0.68, 0.07381037142518096)
(0.7000000000000001, 0.05440844903287054)
(0.7

#DBSCAN

### Создание списка из мини-датасетов

In [0]:
from scipy.sparse import coo_matrix, vstack
def mini_matrix_list(df):
  #define the min and the max date
  min_date = df.sort_values(['date']).reset_index().date[0] #min
  max_date = df.sort_values(['date']).reset_index().date[df.shape[0] - 1] #max
  print(min_date, max_date)

  import datetime
  timedelta = datetime.timedelta(days = 1)
  from datetime import datetime as dt
  def string_into_datetime(day, timedelta):
    return (dt.strptime(day, '%Y-%m-%d') - datetime.timedelta(days = timedelta)).strftime('%Y-%m-%d')

  delta_days = (dt.strptime(max_date, '%Y-%m-%d') - dt.strptime(min_date, '%Y-%m-%d')).days

  mini_datasets_list = [df[df.date == string_into_datetime(max_date, timedelta)].index for timedelta in range(delta_days)]

  sm_list = [(vstack([tfs[elem] for elem in mini_datasets_list[i]]), i) for i in range(delta_days) if len(mini_datasets_list[i]) != 0]

  len(sm_list)
  return mini_datasets_list, sm_list

In [0]:
filtered_df.columns = ['full_text', 'to_id', 'id', 'likes', 'reposts', 'comments', 'date']

In [0]:
from datetime import datetime as dt
def toMyTime(day):
  myTime = dt.strptime(day, "%Y-%m-%d %H:%M:%S")
  myFormat = "%Y-%m-%d"
  day = myTime.strftime(myFormat)
  return day
filtered_df.date = filtered_df.date[:filtered_df.shape[0]].apply(toMyTime)

In [0]:
mini_datasets_list, sm_list = mini_matrix_list(filtered_df)

('2018-06-01', '2018-11-30')


### **Применение DBSCAN для текущего датасета**

In [0]:
from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
import numpy as np
def DBSCAN_(X):
  db = DBSCAN(eps=0.52, min_samples=2, algorithm = 'brute', metric='cosine').fit(X)
  core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
  core_samples_mask[db.core_sample_indices_] = True
  labels = db.labels_
  return labels


In [0]:
df = filtered_df
dataset_list = []
dataset_additional_list = []
num_of_clust = 0
for i in range(len(sm_list)):
  labels = DBSCAN_(sm_list[i][0])
  temp_df = df[df.index.isin(mini_datasets_list[sm_list[i][1]])]
  temp_df['cluster'] = labels
  temp_df = temp_df[temp_df.cluster != -1]
  n_current = temp_df.cluster.nunique()
  num_of_clust += n_current
  for i, k in enumerate(range(num_of_clust - n_current, num_of_clust)):
    temp_df.loc[temp_df.cluster == i, "cluster"] = k
    temp_temp_df = temp_df[temp_df.cluster == k]
    if temp_temp_df.full_text.nunique() == temp_temp_df.to_id.nunique():  
      dataset_list.extend(temp_temp_df.values.tolist())
    else:
      dataset_additional_list.extend(temp_temp_df.values.tolist())

dataset_list.extend(dataset_additional_list)
dataset = pd.DataFrame.from_records(dataset_list, columns=['full_text', 'to_id', 'id', 'likes', 'reposts', 'comments', 'date', 'cluster'])

In [0]:
group_post_list = list()
group_post_list.extend([dataset.to_id[i], dataset.id[i]] for i in range(dataset.shape[0]))
group_post_list
with open('group_list.pkl', 'wb') as f:
  pickle.dump(group_post_list, f)

# Загрузка и фильтр комментариев

## Загрузка комментариев

In [0]:
import math
import time
def getcomments(owner_id, post_id, k):
  try:
    time.sleep(1)
    offset = 0
    comment = vk_api.wall.getComments(owner_id = owner_id, post_id = post_id, count = 100)
    if comment[0] > 100:
      for i in range(1, int(math.ceil(float(comment[0])/100))):
        time.sleep(1)
        offset += 100
        comment.extend(vk_api.wall.getComments(owner_id = owner_id, post_id = post_id, count = 100, offset = offset)[1:])
    print(k)
  except:
    time.sleep(5)
    comment = getcomments(owner_id, post_id, k)
  return comment
comments = list()

with open('group_post_list.pkl', 'rb') as f:
  group_post_list = pickle.load(f)

for k, elem in enumerate(group_post_list):
  comments.append(getcomments(owner_id = elem[0], post_id = elem[1], k=k))

import pickle
with open('comments.part.pkl', 'wb') as f:
  pickle.dump(comments, f)

In [0]:
import pickle
with open('comments.pkl', 'rb') as f:
  comments = pickle.load(f)

In [0]:
#comments = list()
with open('comments.part1.pkl', 'rb') as f:
  comments_part_1 = pickle.load(f)
with open('comments.part2.pkl', 'rb') as f:
  comments_part_2 = pickle.load(f)
with open('comments.part3.pkl', 'rb') as f:
  comments_part_3 = pickle.load(f)
with open('comments.part4.pkl', 'rb') as f:
  comments_part_4 = pickle.load(f)
with open('comments.part5.pkl', 'rb') as f:
  comments_part_5 = pickle.load(f)


In [0]:
comments_temp = list()
comments_temp.extend(comments_part_1)
comments_temp.extend(comments_part_2)
comments_temp.extend(comments_part_3)
comments_temp.extend(comments_part_4)
comments_temp.extend(comments_part_5)

## Предобработка комментариев

In [0]:
def comment_preprocess(text):
    """ Remove links, <*> and additional spaces from text
    """
    #try:
    #  text = text.decode('utf-8')
    #except:
    #  pass
    pattern = re.compile('<[^>]*>')
  #      text = pattern.sub(' ', text)
  #      pattern = re.compile('\(*\)')
    text = pattern.sub(u' ', text)
    pattern = re.compile(u'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    text = pattern.sub(u' ', text)
    text = text.lower().replace(u"ё", u"е")
    text = re.sub(u'[^a-zA-Zа-яА-Я1-9]+', u' ', text)
    pattern = re.compile(u'\s+')
    text = pattern.sub(u' ', text)
    text = ' '.join([i for i in text.split() if i[0] != u'#' and i[0] != u'@' and i[:5] != u'ВИДЕО' and i[:4] != u'ФОТО'])
    return text

In [0]:
comments_updated = list()
for post in comments:
  comm = list()
  for comment in post[1:]:
    try:
      comment[u'reply_to_cid']
      comm.append(comment_preprocess(' '.join(comment[u'text'].split()[1:])))
    except:
      comm.append(comment_preprocess(comment[u'text']))
  comments_updated.append(comm)
    
  
comments.extend(comments_updated)
comments_updated = comments
comments_updated = list()
for post in comments:
  tmp_list = list()
  for comment in post:
    tmp_list.append(comment_preprocess(comment))
  comments_updated.append(tmp_list)
for num, post in enumerate(comments_updated):
  for num_com in range(len(post)-1, -1, -1):
    if post[num_com] == u'':
      comments_updated[num].pop(num_com)

In [0]:
with open('comments.pkl', 'wb') as f:
  pickle.dump(comments_updated, f)

**Лемматизация комментариев**

In [0]:
with open('comments.pkl', 'rb') as f:
  comments_updated = pickle.load(f)

In [0]:
#comments_lemmatized = list()
for k, post in enumerate(comments_updated):
  tmp = list()
  for comment in post:
    tmp.append(pymorphy_lemmatizer(comment))
  comments_lemmatized.append(tmp)
  print(k)
#print(pymorphy_lemmatizer(comments_updated[x][y]))
with open('comments_lemmatized.pkl', 'wb') as f:
  pickle.dump(comments_lemmatized, f)
with open('comments_lemmatized.pkl', 'rb') as f:
  comments_lemmatized = pickle.load(f)

# Обучение наивного байесовского классификатора

In [0]:
# Read the data from CSV files
n = ['id', 'date','name','text','typr','rep','rtw','faw','stcount','foll','frien','listcount']
data_positive = pd.read_csv('positive.csv', sep=';',error_bad_lines=False, names=n, usecols=['text'], encoding='utf-8')
data_negative = pd.read_csv('negative.csv', sep=';',error_bad_lines=False, names=n, usecols=['text'], encoding='utf-8')

In [0]:
# Create balanced dataset
sample_size = min(data_positive.shape[0], data_negative.shape[0])
raw_data = np.concatenate((data_positive['text'].values[:sample_size], 
                           data_negative['text'].values[:sample_size]), axis=0) 
labels = [1]*sample_size + [0]*sample_size

In [0]:
import re

def preprocess_text(text):
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL', text)
    text = re.sub('@[^\s]+','USER', text)
    text = re.sub(' +',' ', text)
    return text.strip()

data = [preprocess_text(t) for t in raw_data]

In [0]:
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV

text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB())])

tuned_parameters = {
    'vect__ngram_range': [(1, 2), (1, 3)],
    'tfidf__use_idf': (True, False),
    'tfidf__norm': ('l1', 'l2'),
    'clf__alpha': [1, 1e-1, 1e-2]
}



In [0]:
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.33, random_state=42)

In [0]:
from sklearn.metrics import classification_report

score = 'f1_macro'
print("# Tuning hyper-parameters for %s" % score)
print()
np.errstate(divide='ignore')
clf = GridSearchCV(text_clf, tuned_parameters, cv=10, scoring=score)
clf.fit(x_train, y_train)

print("Best parameters set found on development set:")
print()
print(clf.best_params_)
print()
print("Grid scores on development set:")
print()
for mean, std, params in zip(clf.cv_results_['mean_test_score'], 
                             clf.cv_results_['std_test_score'], 
                             clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))
print()

print("Detailed classification report:")
print()
print("The model is trained on the full development set.")
print("The scores are computed on the full evaluation set.")
print()
print(classification_report(y_test, clf.predict(x_test), digits=4))
print()

# Tuning hyper-parameters for f1_macro
()
Best parameters set found on development set:
()
{'vect__ngram_range': (1, 3), 'tfidf__use_idf': True, 'tfidf__norm': 'l2', 'clf__alpha': 1}
()
Grid scores on development set:
()
0.743 (+/-0.005) for {'vect__ngram_range': (1, 2), 'tfidf__use_idf': True, 'tfidf__norm': 'l1', 'clf__alpha': 1}
0.741 (+/-0.006) for {'vect__ngram_range': (1, 3), 'tfidf__use_idf': True, 'tfidf__norm': 'l1', 'clf__alpha': 1}
0.733 (+/-0.004) for {'vect__ngram_range': (1, 2), 'tfidf__use_idf': False, 'tfidf__norm': 'l1', 'clf__alpha': 1}
0.729 (+/-0.006) for {'vect__ngram_range': (1, 3), 'tfidf__use_idf': False, 'tfidf__norm': 'l1', 'clf__alpha': 1}
0.755 (+/-0.006) for {'vect__ngram_range': (1, 2), 'tfidf__use_idf': True, 'tfidf__norm': 'l2', 'clf__alpha': 1}
0.756 (+/-0.005) for {'vect__ngram_range': (1, 3), 'tfidf__use_idf': True, 'tfidf__norm': 'l2', 'clf__alpha': 1}
0.753 (+/-0.004) for {'vect__ngram_range': (1, 2), 'tfidf__use_idf': False, 'tfidf__norm': 'l2', 'c

In [0]:
with open('clf.pkl', 'wb') as f:
    pickle.dump(clf, f, protocol=2)

# Анализ манипулирования откликом

## Фильтр комментариев

In [0]:
with open('comments.pkl', 'rb') as f:
  comments = pickle.load(f)

In [0]:
del_com_list = []
for post_num, post in enumerate(comments):
  for comment_num, comment in enumerate(post):
        if len(comment) < 40 or len(comment) > 300 or comment == u'Комментарий удалён пользователем или руководителем страницы':
          del_com_list.append([post_num, comment_num])
del_com_list = sorted(del_com_list, key = lambda y: (y[0], -y[1]))


for elem in del_com_list:
  comments[elem[0]].pop(elem[1])

for post_num in range(len(comments)-1, -1, -1):
  if len(comments[post_num]) < 10:
        comments.pop(post_num)
        dataset = dataset.drop([post_num])

In [0]:
# Remove clusters having less than 2 posts
dataset = dataset.reset_index(drop=True)
for clust_num in dataset.cluster.get_values():
  tmp = dataset[dataset.cluster == clust_num]
  if tmp.shape[0] < 2:
    values = tmp.index
    for value in sorted(values, reverse = True):
      del comments[value]
    dataset = dataset.drop(values).reset_index(drop=True)
    

## Подсчет Bias

In [0]:
with open('clf.pkl', 'rb') as f:
  clf = pickle.load(f)

In [0]:
def clf_pred(comments):
  pred = [(clf.predict_proba([comment])[0][1] - 0.5)*2 for comment in comments]
  return sum(pred)/len(pred)

In [0]:
import re

def preprocess_com(text):
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL', text)
    text = re.sub('@[^\s]+','USER', text)
    text = text.lower().replace("ё", "е")
    text = re.sub('[^a-zA-Zа-яА-Я1-9]+', ' ', text)
    text = re.sub(' +',' ', text)
    return text.strip()


In [0]:
cluster_dataset = pd.DataFrame(index = range(dataset.cluster.nunique()), columns = dataset.to_id.unique())
#cluster_list = []
for k, cluster in enumerate(dataset.cluster.unique()):
  tmp_dict = {str(column):[] for column in cluster_dataset.columns}
  tmp_ds = dataset[dataset.cluster == cluster]
  for index in tmp_ds.index:
#    cluster_list.append([])
    #if len(tmp_ds[dataset.to_id == dataset[index].to_id]) > 1:
    tmp_dict[str(dataset.to_id[index])].append(clf_pred(comments[index]))
  for key, value in tmp_dict.iteritems():
    cluster_dataset[int(key)][k] = sum(value)/len(value) if len(value) != 0 else value
  print(k)
  

In [0]:
avg = cluster_dataset.mean(axis=1)
bias_dataset = pd.DataFrame(index = range(dataset.cluster.nunique()), columns = dataset.to_id.unique())
for k, elem in enumerate(avg):
  for id in dataset.to_id.unique():
    bias_dataset[id][k] = cluster_dataset[id][k] - elem
  print(k)


In [0]:
import numpy as np
for column in cluster_dataset.columns:
  for row in range(cluster_dataset.shape[0]):
    if cluster_dataset[column][row] == []:
      cluster_dataset[column][row] = np.nan

In [0]:
cluster_dataset

,-26493942,-67991642,-23482909,-76982440,-15755094,-106879986,-27532693,-25232578,-20648295,-20169232,-23304496
0,0.0975019,0.0320194,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-0.0120111,NaN,0.115616,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-0.0172142,0.0518887,NaN,0.107506,0.0832876,0.07524,0.110514,NaN,NaN,NaN,NaN
3,0.0401869,0.0155486,NaN,NaN,NaN,0.037635,NaN,NaN,NaN,NaN,NaN
4,0.142381,NaN,NaN,NaN,0.0466894,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,0.143737,0.118825,NaN,NaN,0.00936346,NaN,NaN,NaN
6,0.0522343,NaN,NaN,0.0949451,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,0.19992,0.130695,0.0625058,NaN,NaN,NaN,NaN,0.0747609,NaN,NaN,NaN
8,NaN,-0.051818,NaN,NaN,NaN,0.0107481,NaN,NaN,NaN,NaN,NaN
9,NaN,0.0672197,NaN,NaN,NaN,-0.0137032,NaN,NaN,NaN,NaN,NaN


In [0]:
import pickle
with open('cluster_dataset.pkl', 'wb') as f:
  pickle.dump(cluster_dataset, f)

In [0]:
bias_dataset

,-26493942,-67991642,-23482909,-76982440,-15755094,-106879986,-27532693,-25232578,-20648295,-20169232,-23304496
0,0.0327412,-0.0327412,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-0.0638136,NaN,0.0638136,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-0.0857513,-0.0166483,NaN,0.0389686,0.0147505,0.00670293,0.0419774,NaN,NaN,NaN,NaN
3,0.00906338,-0.0155749,NaN,NaN,NaN,0.00651151,NaN,NaN,NaN,NaN,NaN
4,0.0478457,NaN,NaN,NaN,-0.0478457,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,0.0530953,0.028183,NaN,NaN,-0.0812783,NaN,NaN,NaN
6,-0.0213554,NaN,NaN,0.0213554,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,0.0829494,0.0137245,-0.0544645,NaN,NaN,NaN,NaN,-0.0422094,NaN,NaN,NaN
8,NaN,-0.031283,NaN,NaN,NaN,0.031283,NaN,NaN,NaN,NaN,NaN
9,NaN,0.0404615,NaN,NaN,NaN,-0.0404615,NaN,NaN,NaN,NaN,NaN


In [0]:
public_list = [['РЕН ТВ', 26493942],['Лента.ру', 67991642], ['Медуза', 76982440],
               ['Коммерсантъ', 23482909], ['РБК', 25232578], ['РИА Новости', 15755094], ['Mail.ru Новости', 106879986],
               ['Газета.Ru', 20169232], ['Российская газета', 23304496], ['IZ.RU', 27532693], ['Московский Комсомолец (МК)', 20648295]]
column_list = []
for column in bias_dataset.columns:
  for elem in public_list:
    if int(column) == -elem[1]:
      column_list.append(elem[0])
bias_dataset.columns = column_list

In [0]:
bias_dataset.mean(axis=0)*1000

РЕН ТВ                       -8.100047
Лента.ру                      1.297006
Коммерсантъ                   2.246590
Медуза                        0.437009
РИА Новости                  -0.403877
Mail.ru Новости               3.948076
IZ.RU                        -2.440344
РБК                           1.379841
Московский Комсомолец (МК)    0.461154
Газета.Ru                     5.151802
Российская газета            -2.043316
dtype: float64

In [0]:
bias_dataset[bias_dataset < 0].mean(axis=0) * 100

РЕН ТВ                       -4.455712
Лента.ру                     -3.435338
Коммерсантъ                  -3.736611
Медуза                       -3.304558
РИА Новости                  -3.121703
Mail.ru Новости              -3.173461
IZ.RU                        -4.272266
РБК                          -3.104626
Московский Комсомолец (МК)   -4.994420
Газета.Ru                    -4.547292
Российская газета            -4.576182
dtype: float64

In [0]:
bias_dataset[bias_dataset > 0].mean(axis=0) * 100

РЕН ТВ                        3.642454
Лента.ру                      3.634485
Коммерсантъ                   3.770170
Медуза                        3.291461
РИА Новости                   3.145984
Mail.ru Новости               3.523015
IZ.RU                         4.063332
РБК                           3.089296
Московский Комсомолец (МК)    4.980055
Газета.Ru                     4.525484
Российская газета             4.332674
dtype: float64